# Week 2: Surface Energy Balance Practical 

Module: GEOL0022 Ocean Physics and Climate Change

Author: Michel Tsamados 

In this practical we will learn to: 
- Download data from ERA5
- Plot fluxes on a map 
- Perform some simple analysis on the data


This is not an introduction to python and we will learn on the go. For those who want to learn more systematically here is a list of recommended tutorials selected by William Gregory


http://mathesaurus.sourceforge.net/matlab-numpy.html 

http://swcarpentry.github.io/python-novice-inflammation/

https://medium.freecodecamp.org/why-you-need-python-environments-and-how-to-manage-them-with-conda-85f155f4353c

https://github.com/nansencenter/nansat-lectures

https://jeffknupp.com/blog/2014/06/18/improve-your-python-python-classes-and-object-oriented-programming/

There’s also some good YouTube videos out there describing how to programme ‘object-orientedly’ in Python too…

# Import packages

Lets get started. First we need to import some tools. 

All the commands below are for importing packages that will be used in this practical.  

In [2]:
! pip install cftime==1.4.1

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
import sys, os
# sys.path.insert(0, os.path.expanduser('~/.local/lib/python3.8/site-packages'))
from netCDF4 import Dataset as netcdf 
from scipy import signal
# import cdsapi
from pylab import *
import numpy as np
import os
import matplotlib.pyplot as plt
import datetime
import scipy.stats as stats
import cftime
from cftime import utime
import zipfile
import warnings
import pandas as pd
import statsmodels.api as sm
warnings.simplefilter('ignore')
import matplotlib.dates as mdateschan
from matplotlib.dates import MonthLocator, WeekdayLocator, DateFormatter
import matplotlib.ticker as ticker
from matplotlib.pylab import rcParams
import matplotlib.dates as mdates
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import zipfile
import xarray as xr


In [2]:
# import cdsapi


# Download some data to play with 

Data download from Copernicus/ECMWF portal
https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=form

In [3]:
# import cdsapi

# c = cdsapi.Client()

# c.retrieve(
#     'reanalysis-era5-single-levels-monthly-means',
#     {
#         'product_type': 'monthly_averaged_reanalysis',
#         'variable': [
#             'clear_sky_direct_solar_radiation_at_surface', 'downward_uv_radiation_at_the_surface', 'forecast_logarithm_of_surface_roughness_for_heat',
#             'instantaneous_surface_sensible_heat_flux', 'near_ir_albedo_for_diffuse_radiation', 'near_ir_albedo_for_direct_radiation',
#             'surface_latent_heat_flux', 'surface_net_solar_radiation', 'surface_net_solar_radiation_clear_sky',
#             'surface_net_thermal_radiation', 'surface_net_thermal_radiation_clear_sky', 'surface_sensible_heat_flux',
#             'surface_solar_radiation_downward_clear_sky', 'surface_solar_radiation_downwards', 'surface_thermal_radiation_downward_clear_sky',
#             'surface_thermal_radiation_downwards', 'toa_incident_solar_radiation', 'top_net_solar_radiation',
#             'top_net_solar_radiation_clear_sky', 'top_net_thermal_radiation', 'top_net_thermal_radiation_clear_sky',
#             'total_sky_direct_solar_radiation_at_surface', 'uv_visible_albedo_for_diffuse_radiation', 'uv_visible_albedo_for_direct_radiation',
#         ],
#         'year': '2019',
#         'month': [
#               '01',
# #             '01', '02', '03',
# #             '04', '05', '06',
# #             '07', '08', '09',
# #             '10', '11', '12',
#         ],
#         'time': '00:00',
#         'format': 'netcdf',
#     },
#     './Fluxes_2019.nc')
# # 	'../../data/week2/Fluxes_2019.nc')

2021-12-16 17:13:07,056 INFO Welcome to the CDS
2021-12-16 17:13:07,058 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2021-12-16 17:13:07,228 INFO Request is completed
2021-12-16 17:13:07,229 INFO Downloading https://download-0006.copernicus-climate.eu/cache-compute-0006/cache/data7/adaptor.mars.internal-1639673771.2697363-20583-19-798effeb-6ba3-4ae7-bb55-097da27a1b5f.nc to ./Fluxes_2019.nc (47.5M)
2021-12-16 17:13:10,851 INFO Download rate 13.1M/s  


Result(content_length=49851868,content_type=application/x-netcdf,location=https://download-0006.copernicus-climate.eu/cache-compute-0006/cache/data7/adaptor.mars.internal-1639673771.2697363-20583-19-798effeb-6ba3-4ae7-bb55-097da27a1b5f.nc)

In [2]:
! ls -lrt 

total 43383
-rw-r--r-- 1 user user 24927560 Dec 14  2021 land_mask.nc
-rw-r--r-- 1 user user 49851868 Dec 16  2021 Fluxes_2019.nc
-rw-r--r-- 1 user user    75613 Oct 13 13:11 Practical1.ipynb


In [3]:
# import cdsapi

# c = cdsapi.Client()

# c.retrieve(
#     'reanalysis-era5-single-levels-monthly-means',
#     {
#         'product_type': 'monthly_averaged_reanalysis',
#         'variable': 'land_sea_mask',
#         'year': '2019',
#         'month': [
#             '01', '02', '03',
#             '04', '05', '06',
#             '07', '08', '09',
#             '10', '11', '12',
#         ],
#         'time': '00:00',
#         'format': 'netcdf',
#     },
#     '../../data/week2/land_mask.nc')

2020-10-23 09:18:18,578 INFO Welcome to the CDS
2020-10-23 09:18:18,579 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2020-10-23 09:18:19,464 INFO Request is queued
2020-10-23 09:18:22,160 INFO Request is running
2020-10-23 09:18:27,983 INFO Request is completed
2020-10-23 09:18:27,985 INFO Downloading http://136.156.133.39/cache-compute-0012/cache/data3/adaptor.mars.internal-1603444701.141617-23815-10-f3dd9cf1-717e-47f3-b50a-9ec0e07879a1.nc to ../../data/week2/land_mask.nc (23.8M)
2020-10-23 09:18:30,509 INFO Download rate 9.4M/s   


Result(content_length=24927560,content_type=application/x-netcdf,location=http://136.156.133.39/cache-compute-0012/cache/data3/adaptor.mars.internal-1603444701.141617-23815-10-f3dd9cf1-717e-47f3-b50a-9ec0e07879a1.nc)

Some simple command line commands. ls to check what files are in the runnind directory 

In [2]:
# ! ls ../../data/week2

Fluxes_1979.nc	Fluxes_1979_01.nc  Fluxes_2019.nc  land_mask.nc


pwd to check in which directory you are currently 

In [3]:
# ! pwd

/home/user/notes/week2


# Xarray structures 

We can load the data into the more advanced and versatile xarray structure. 

For a 45 minutes intro on what these are read here https://xarray-contrib.github.io/xarray-tutorial/oceanhackweek-2020/xarray-oceanhackweek20.html

In [9]:
# #load data
# import xarray as xr
# ds_1979 = xr.open_dataset('../../data/week2/Fluxes_1979.nc')
# print(ds_1979)

In [1]:
# ! ls ..

In [ ]:
#Get Fluxes_2019.nc
! gdown --id 1bI-7ZLCCkVPNB2bAXeDg9qPnmGj_pOIy

In [ ]:
#Get land_mask.nc
! gdown --id 1bHCC6pQaxHKJjHEE9BRSLvSGGY-Cmv0e

In [2]:
#load data
import xarray as xr
ds_2019 = xr.open_dataset('./Fluxes_2019.nc')
mask_2019 = xr.open_dataset('./land_mask.nc')
print(ds_2019)

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 1)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 2019-01-01
Data variables:
    cdir       (time, latitude, longitude) float32 ...
    uvb        (time, latitude, longitude) float32 ...
    flsr       (time, latitude, longitude) float32 ...
    ishf       (time, latitude, longitude) float32 ...
    alnid      (time, latitude, longitude) float32 ...
    alnip      (time, latitude, longitude) float32 ...
    slhf       (time, latitude, longitude) float32 ...
    ssr        (time, latitude, longitude) float32 ...
    ssrc       (time, latitude, longitude) float32 ...
    str        (time, latitude, longitude) float32 ...
    strc       (time, latitude, longitude) float32 ...
    sshf       (time, latitude, longitude) float32 ...
    ssrdc      (time, latitud

In [5]:
#lons lats on a grid
[lons, lats] = meshgrid(ds_2019.longitude, ds_2019.latitude)

Variables are defined here

https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=overview

https://confluence.ecmwf.int/pages/viewpage.action?pageId=82870405

In [5]:
print(ds_2019.sshf)

<xarray.DataArray 'sshf' (time: 1, latitude: 721, longitude: 1440)>
[1038240 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 2019-01-01
Attributes:
    units:          J m**-2
    long_name:      Surface sensible heat flux
    standard_name:  surface_upward_sensible_heat_flux


# Plotting 

Now we are ready to plot some nice maps with the fluxes that we have downloaded from ECMWF

## First plot with a basic colorbar

In [1]:
#plot data on other projections

year_str='2019'
month_str='01'

# t2m_test = ds.t2m.sel(time='1979-01-01', method='nearest')
field_test=(ds_2019.sshf).sel(time=year_str+'-'+month_str+'-01', method='nearest')
# field_test=ds_2019.sshf.sum(dim='time') #yearly average
# field_test=((ds_2019.sshf)/(31.*24.*3600.)).sel(time=year_str+'-'+month_str+'-01', method='nearest')
plt.figure()
ax = plt.axes(projection=ccrs.Mercator())
#     ax.stock_img()
ax.coastlines()


fig = plt.figure(figsize=(18,12))
#     ax = plt.axes(projection=ccrs.Orthographic())
ax.coastlines()
ax.gridlines()
# field_test.plot(ax=ax, transform=ccrs.PlateCarree(), cbar_kwargs={'shrink': 0.4})
(field_test/86400.).plot(ax=ax, transform=ccrs.PlateCarree(), cbar_kwargs={'shrink': 0.4})


ax.set_title('Surface Sensible Heat Flux in January 2019 (W/m^2) ')


NameError: name 'ds_2019' is not defined

In [8]:
# biasContDist=10
# # iMinMax=-120
# iMinMax=12.*biasContDist/2.+biasContDist/2.
# clevsTD=np.arange(-2*iMinMax,+0.0001,biasContDist)
# clevsTD=np.arange(-iMinMax,iMinMax+0.0001,biasContDist)
print(clevsTD)

NameError: name 'clevsTD' is not defined

## Surface Sensible Heat Flux (sshf)

In [7]:
#plot data on other projections 
#https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html
plt.clf()
year_str='2019'
month_str='01'

# t2m_test = ds.t2m.sel(time='1979-01-01', method='nearest')
field_test=(ds_2019.sshf).sel(time=year_str+'-'+month_str+'-01', method='nearest')

# t2m_test = ds.t2m.sel(time='1979-01-01', method='nearest')
# field_test=(ds_2019.sshf).sel(time=year_str+'-'+month_str+'-01', method='nearest')
# field_test=ds_2019.slhf.mean(dim='time') #yearly average
# field_test=((ds_2019.sshf)/(31.*24.*3600.)).sel(time=year_str+'-'+month_str+'-01', method='nearest')
plt.figure()
ax = plt.axes(projection=ccrs.Mercator(central_longitude=180))
#     ax.stock_img()
ax.coastlines()

biasContDist=10
# iMinMax=-120
iMinMax=12.*biasContDist/2.+biasContDist/2.
clevsTD=np.arange(-2*iMinMax,+0.0001,biasContDist)

fig = plt.figure(figsize=(24,12))
#     ax = plt.axes(projection=ccrs.Orthographic())
ax.coastlines()
ax.gridlines()
(field_test/86400.).plot.contourf(ax=ax, transform=ccrs.PlateCarree(), clevel=clevsTD, extend='both', cmap='coolwarm', vmin=-130, vmax=0, cbar_kwargs={'shrink': 0.4})


ax.set_title('Surface Sensible Heat Flux in January 2019 (W/m^2) ')


## Surface Latent Heat Flux (slhf) selected month (i.e. January 2019)

In [8]:
#plot data on other projections 
#https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html
plt.clf()

# t2m_test = ds.t2m.sel(time='1979-01-01', method='nearest')
# field_test=(ds_2019.sshf).sel(time=year_str+'-'+month_str+'-01', method='nearest')
field_test=(ds_2019.sshf).sel(time=year_str+'-'+month_str+'-01', method='nearest')
# field_test=((ds_2019.sshf)/(31.*24.*3600.)).sel(time=year_str+'-'+month_str+'-01', method='nearest')
plt.figure()
ax = plt.axes(projection=ccrs.Mercator(central_longitude=180))
#     ax.stock_img()
ax.coastlines()

biasContDist=20
# iMinMax=-120
iMinMax=12.*biasContDist/2.+biasContDist/2.
clevsTD=np.arange(-2*iMinMax,+0.0001,biasContDist)

fig = plt.figure(figsize=(18,12))
#     ax = plt.axes(projection=ccrs.Orthographic())
ax.coastlines()
ax.gridlines()
(field_test/86400.).plot.contourf(ax=ax, transform=ccrs.PlateCarree(), clevel=clevsTD, extend='both', cmap='coolwarm', vmin=-260, vmax=0, cbar_kwargs={'shrink': 0.4})


ax.set_title('Surface Latent Heat Flux for January 2019 (W/m^2)')


ERROR! Session/line number was not unique in database. History logging moved to new session 19


Text(0.5, 1.0, 'Surface Latent Heat Flux for January 2019 (W/m^2)')

<Figure size 864x504 with 0 Axes>

<Figure size 1296x864 with 0 Axes>

## The Bowden ration = Qh / Qe = sshf / slhf

In [6]:
#plot data on other projections 
#https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html
plt.clf()
year_str='2019'
month_str='07'

# t2m_test = ds.t2m.sel(time='1979-01-01', method='nearest')
# field_test=(ds_2019.sshf).sel(time=year_str+'-'+month_str+'-01', method='nearest')
field_test1=(ds_2019.sshf).sel(time=year_str+'-'+month_str+'-01', method='nearest')
field_test2=(ds_2019.slhf).sel(time=year_str+'-'+month_str+'-01', method='nearest')
# field_test=((ds_2019.sshf)/(31.*24.*3600.)).sel(time=year_str+'-'+month_str+'-01', method='nearest')
plt.figure()
ax = plt.axes(projection=ccrs.Mercator(central_longitude=180))
#     ax.stock_img()
ax.coastlines()

biasContDist=0.1
# iMinMax=-120
iMinMax=12.*biasContDist/2.+biasContDist/2.
clevsTD=np.arange(0,2*iMinMax+0.0001,biasContDist)

fig = plt.figure(figsize=(18,12))
#     ax = plt.axes(projection=ccrs.Orthographic())
ax.coastlines()
ax.gridlines()
(field_test1/field_test2).plot.contourf(ax=ax, transform=ccrs.PlateCarree(), clevel=clevsTD, extend='both', cmap='coolwarm', vmin=0, vmax=1, cbar_kwargs={'shrink': 0.4})


ax.set_title('Bowen ratio R = Qh/Qe for July 2019 (W/m^2)')


Text(0.5, 1.0, 'Bowen ratio R = Qh/Qe for July 2019 (W/m^2)')

<Figure size 864x504 with 0 Axes>

<Figure size 1296x864 with 0 Axes>

## Surface fluxes for the whole year 2019

In [10]:
#plot data on other projections 
#https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html
plt.clf()

# t2m_test = ds.t2m.sel(time='1979-01-01', method='nearest')
# field_test=(ds_2019.sshf).sel(time=year_str+'-'+month_str+'-01', method='nearest')
field_test=ds_2019.slhf.mean(dim='time') #yearly average
# field_test=((ds_2019.sshf)/(31.*24.*3600.)).sel(time=year_str+'-'+month_str+'-01', method='nearest')
plt.figure()
ax = plt.axes(projection=ccrs.Mercator(central_longitude=180))
#     ax.stock_img()
ax.coastlines()

biasContDist=20
# iMinMax=-120
iMinMax=12.*biasContDist/2.+biasContDist/2.
clevsTD=np.arange(-2*iMinMax,+0.0001,biasContDist)

fig = plt.figure(figsize=(18,12))
#     ax = plt.axes(projection=ccrs.Orthographic())
ax.coastlines()
ax.gridlines()
(field_test/86400.).plot.contourf(ax=ax, transform=ccrs.PlateCarree(), clevel=clevsTD, extend='both', cmap='coolwarm', vmin=-260, vmax=0, cbar_kwargs={'shrink': 0.4})


ax.set_title('Surface Latent Heat Flux for 2019')
    

In [44]:
#plot data on other projections
plt.clf()

# t2m_test = ds.t2m.sel(time='1979-01-01', method='nearest')
# field_test=(ds_2019.sshf).sel(time=year_str+'-'+month_str+'-01', method='nearest')
field_test=ds_2019.sshf.mean(dim='time') #yearly average
# field_test=((ds_2019.sshf)/(31.*24.*3600.)).sel(time=year_str+'-'+month_str+'-01', method='nearest')
plt.figure()
ax = plt.axes(projection=ccrs.Mercator(central_longitude=180))
#     ax.stock_img()
ax.coastlines()

biasContDist=10
# iMinMax=-120
iMinMax=12.*biasContDist/2.+biasContDist/2.
clevsTD=np.arange(-2*iMinMax,+0.0001,biasContDist)

fig = plt.figure(figsize=(18,12))
#     ax = plt.axes(projection=ccrs.Orthographic())
ax.coastlines()
ax.gridlines()
(field_test/86400.).plot.contourf(ax=ax, transform=ccrs.PlateCarree(), clevel=clevsTD, extend='both', cmap='coolwarm', vmin=-130, vmax=0, cbar_kwargs={'shrink': 0.4})


ax.set_title('Surface Sensible Heat Flux for 2019 (W/m^2)')


Text(0.5, 1.0, 'Surface Sensible Heat Flux for 2019 (W/m^2)')

<Figure size 864x504 with 0 Axes>

<Figure size 1296x864 with 0 Axes>

In [47]:
#plot data on other projections
plt.clf()

# t2m_test = ds.t2m.sel(time='1979-01-01', method='nearest')
# field_test=(ds_2019.sshf).sel(time=year_str+'-'+month_str+'-01', method='nearest')
field_test=ds_2019.ssr.mean(dim='time') #yearly average
# field_test=((ds_2019.sshf)/(31.*24.*3600.)).sel(time=year_str+'-'+month_str+'-01', method='nearest')
plt.figure()
ax = plt.axes(projection=ccrs.Mercator(central_longitude=180))
#     ax.stock_img()
ax.coastlines()

biasContDist=10
# iMinMax=-120
iMinMax=12.*biasContDist/2.+biasContDist/2.
# clevsTD=np.arange(-2*iMinMax,+0.0001,biasContDist)
clevsTD=np.arange(0.,+2*iMinMax+0.0001,biasContDist)

fig = plt.figure(figsize=(18,12))
#     ax = plt.axes(projection=ccrs.Orthographic())
ax.coastlines()
ax.gridlines()
(field_test/86400.).plot.contourf(ax=ax, transform=ccrs.PlateCarree(), clevel=clevsTD, extend='both', cmap='coolwarm', vmin=0, vmax=130, cbar_kwargs={'shrink': 0.4})


ax.set_title('Surface Net Solar Radiation for 2019 (W/m^2)')


Text(0.5, 1.0, 'Surface Net Solar Radiation for 2019 (W/m^2)')

<Figure size 864x504 with 0 Axes>

<Figure size 1296x864 with 0 Axes>

In [48]:
#plot data on other projections
plt.clf()

# t2m_test = ds.t2m.sel(time='1979-01-01', method='nearest')
# field_test=(ds_2019.sshf).sel(time=year_str+'-'+month_str+'-01', method='nearest')
field_test=ds_2019.str.mean(dim='time') #yearly average
# field_test=((ds_2019.sshf)/(31.*24.*3600.)).sel(time=year_str+'-'+month_str+'-01', method='nearest')
plt.figure()
ax = plt.axes(projection=ccrs.Mercator(central_longitude=180))
#     ax.stock_img()
ax.coastlines()

biasContDist=10
# iMinMax=-120
iMinMax=12.*biasContDist/2.+biasContDist/2.
# clevsTD=np.arange(-2*iMinMax,+0.0001,biasContDist)
clevsTD=np.arange(-2*iMinMax,0.0001,biasContDist)

fig = plt.figure(figsize=(18,12))
#     ax = plt.axes(projection=ccrs.Orthographic())
ax.coastlines()
ax.gridlines()
(field_test/86400.).plot.contourf(ax=ax, transform=ccrs.PlateCarree(), clevel=clevsTD, extend='both', cmap='coolwarm', vmin=-130, vmax=0, cbar_kwargs={'shrink': 0.4})


ax.set_title('Surface Net Thermal Radiation for 2019 (W/m^2)')


Text(0.5, 1.0, 'Surface Net Thermal Radiation for 2019 (W/m^2)')

<Figure size 864x504 with 0 Axes>

<Figure size 1296x864 with 0 Axes>

In [50]:
#plot data on other projections
plt.clf()

# t2m_test = ds.t2m.sel(time='1979-01-01', method='nearest')
# field_test=(ds_2019.sshf).sel(time=year_str+'-'+month_str+'-01', method='nearest')
field_test=ds_2019.str.mean(dim='time')+ds_2019.ssr.mean(dim='time')+ds_2019.sshf.mean(dim='time')+ds_2019.slhf.mean(dim='time')
# field_test=((ds_2019.sshf)/(31.*24.*3600.)).sel(time=year_str+'-'+month_str+'-01', method='nearest')
plt.figure()
ax = plt.axes(projection=ccrs.Mercator(central_longitude=180))
#     ax.stock_img()
ax.coastlines()

biasContDist=10
# iMinMax=-120
iMinMax=12.*biasContDist/2.+biasContDist/2.
# clevsTD=np.arange(-2*iMinMax,+0.0001,biasContDist)
clevsTD=np.arange(-iMinMax,iMinMax+0.0001,biasContDist)

fig = plt.figure(figsize=(18,12))
#     ax = plt.axes(projection=ccrs.Orthographic())
ax.coastlines()
ax.gridlines()
(field_test/86400.).plot.contourf(ax=ax, transform=ccrs.PlateCarree(), clevel=clevsTD, extend='both', cmap='coolwarm', vmin=-65, vmax=65, cbar_kwargs={'shrink': 0.4})


ax.set_title('Surface Net budget for 2019 (W/m^2)')


Text(0.5, 1.0, 'Surface Net budget for 2019 (W/m^2)')

<Figure size 864x504 with 0 Axes>

<Figure size 1296x864 with 0 Axes>

Compare with 1979

In [12]:
#plot data on other projections
plt.clf()

# t2m_test = ds.t2m.sel(time='1979-01-01', method='nearest')
# field_test=(ds_2019.sshf).sel(time=year_str+'-'+month_str+'-01', method='nearest')
field_test2=ds_1979.str.mean(dim='time')+ds_1979.ssr.mean(dim='time')+ds_1979.sshf.mean(dim='time')+ds_1979.slhf.mean(dim='time')
# field_test=((ds_2019.sshf)/(31.*24.*3600.)).sel(time=year_str+'-'+month_str+'-01', method='nearest')
plt.figure()
ax = plt.axes(projection=ccrs.Mercator(central_longitude=180))
#     ax.stock_img()
ax.coastlines()

biasContDist=10
# iMinMax=-120
iMinMax=12.*biasContDist/2.+biasContDist/2.
# clevsTD=np.arange(-2*iMinMax,+0.0001,biasContDist)
clevsTD=np.arange(-iMinMax,iMinMax+0.0001,biasContDist)

fig = plt.figure(figsize=(18,12))
#     ax = plt.axes(projection=ccrs.Orthographic())
ax.coastlines()
ax.gridlines()
(field_test2/86400.).plot.contourf(ax=ax, transform=ccrs.PlateCarree(), clevel=clevsTD, extend='both', cmap='coolwarm', vmin=-65, vmax=65, cbar_kwargs={'shrink': 0.4})


ax.set_title('Surface Net budget for 1979 (W/m^2)')


Text(0.5, 1.0, 'Surface Net budget for 1979 (W/m^2)')

<Figure size 864x504 with 0 Axes>

<Figure size 1296x864 with 0 Axes>

In [13]:
#plot data on other projections
plt.clf()
field_test=ds_2019.str.mean(dim='time')+ds_2019.ssr.mean(dim='time')+ds_2019.sshf.mean(dim='time')+ds_2019.slhf.mean(dim='time')
field_test2=ds_1979.str.mean(dim='time')+ds_1979.ssr.mean(dim='time')+ds_1979.sshf.mean(dim='time')+ds_1979.slhf.mean(dim='time')

plt.figure()
ax = plt.axes(projection=ccrs.Mercator(central_longitude=180))
#     ax.stock_img()
ax.coastlines()

biasContDist=2
# iMinMax=-120
iMinMax=12.*biasContDist/2.+biasContDist/2.
# clevsTD=np.arange(-2*iMinMax,+0.0001,biasContDist)
clevsTD=np.arange(-iMinMax,iMinMax+0.0001,biasContDist)

fig = plt.figure(figsize=(18,12))
#     ax = plt.axes(projection=ccrs.Orthographic())
ax.coastlines()
ax.gridlines()
((field_test-field_test2)/86400.).plot.contourf(ax=ax, transform=ccrs.PlateCarree(), clevel=clevsTD, extend='both', cmap='coolwarm', vmin=-13, vmax=13, cbar_kwargs={'shrink': 0.4})


ax.set_title('Difference in Surface Net budget 2019 - 1979 (W/m^2)')


Text(0.5, 1.0, 'Difference in Surface Net budget 2019 - 1979 (W/m^2)')

<Figure size 864x504 with 0 Axes>

<Figure size 1296x864 with 0 Axes>

# Projections 

Change the projection 

https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html

In [31]:
#plot data on other projections
plt.clf()

# t2m_test = ds.t2m.sel(time='1979-01-01', method='nearest')
# field_test=(ds_2019.sshf).sel(time=year_str+'-'+month_str+'-01', method='nearest')
field_test=ds_2019.str.mean(dim='time')+ds_2019.ssr.mean(dim='time')+ds_2019.sshf.mean(dim='time')+ds_2019.slhf.mean(dim='time')
# field_test=((ds_2019.sshf)/(31.*24.*3600.)).sel(time=year_str+'-'+month_str+'-01', method='nearest')
plt.figure()
# ax = plt.axes(projection=ccrs.Mercator(central_longitude=180))
# ax = plt.axes(projection=ccrs.NorthPolarStereo())
ax = plt.axes(projection=ccrs.Orthographic(0, 90))


#     ax.stock_img()
ax.coastlines()

biasContDist=10
# iMinMax=-120
iMinMax=12.*biasContDist/2.+biasContDist/2.
# clevsTD=np.arange(-2*iMinMax,+0.0001,biasContDist)
clevsTD=np.arange(-iMinMax,iMinMax+0.0001,biasContDist)

fig = plt.figure(figsize=(18,12))
#     ax = plt.axes(projection=ccrs.Orthographic())
ax.coastlines()
ax.gridlines()
(field_test/86400.).plot.contourf(ax=ax, transform=ccrs.PlateCarree(), clevel=clevsTD, extend='both', cmap='coolwarm', vmin=-65, vmax=65, cbar_kwargs={'shrink': 0.4})


ax.set_title('Surface Net budget for 2019 (W/m^2)')


Text(0.5, 1.0, 'Surface Net budget for 2019 (W/m^2)')

<Figure size 864x504 with 0 Axes>

<Figure size 1296x864 with 0 Axes>

In [32]:
#plot data on other projections
plt.clf()

# t2m_test = ds.t2m.sel(time='1979-01-01', method='nearest')
# field_test=(ds_2019.sshf).sel(time=year_str+'-'+month_str+'-01', method='nearest')
field_test=ds_2019.str.mean(dim='time')+ds_2019.ssr.mean(dim='time')+ds_2019.sshf.mean(dim='time')+ds_2019.slhf.mean(dim='time')
# field_test=((ds_2019.sshf)/(31.*24.*3600.)).sel(time=year_str+'-'+month_str+'-01', method='nearest')
plt.figure()
# ax = plt.axes(projection=ccrs.Mercator(central_longitude=180))
# ax = plt.axes(projection=ccrs.NorthPolarStereo())
ax = plt.axes(projection=ccrs.Orthographic(0, 90))


#     ax.stock_img()
ax.coastlines()

biasContDist=10
# iMinMax=-120
iMinMax=12.*biasContDist/2.+biasContDist/2.
# clevsTD=np.arange(-2*iMinMax,+0.0001,biasContDist)
clevsTD=np.arange(-iMinMax,iMinMax+0.0001,biasContDist)

fig = plt.figure(figsize=(18,12))
#     ax = plt.axes(projection=ccrs.Orthographic())
ax.coastlines()
ax.gridlines()
(field_test/86400.).plot.contourf(ax=ax, transform=ccrs.PlateCarree(), clevel=clevsTD, extend='both', cmap='coolwarm', vmin=-65, vmax=65, cbar_kwargs={'shrink': 0.4})


ax.set_title('Surface Net budget for 2019 (W/m^2)')


Text(0.5, 1.0, 'Surface Net budget for 2019 (W/m^2)')

<Figure size 864x504 with 0 Axes>

<Figure size 1296x864 with 0 Axes>

# Some analysis of the data 

Analyse average as a function of latitude

https://xarray-contrib.github.io/xarray-tutorial/oceanhackweek-2020/xarray-oceanhackweek20.html

In [8]:
field_test=ds_2019.str.mean(dim='time')+ds_2019.ssr.mean(dim='time')+ds_2019.sshf.mean(dim='time')+ds_2019.slhf.mean(dim='time')
print(field_test)

<xarray.DataArray (latitude: 721, longitude: 1440)>
array([[-785589.06 , -785589.06 , -785589.06 , ..., -785589.06 ,
        -785589.06 , -785589.06 ],
       [-857885.4  , -857833.2  , -857607.9  , ..., -858250.75 ,
        -858143.75 , -857935.4  ],
       [-910079.94 , -909728.9  , -909421.25 , ..., -910620.75 ,
        -910498.25 , -910599.06 ],
       ...,
       [  10992.709,   10908.709,   10850.334, ...,   11013.334,
          11017.334,   11014.834],
       [   7172.416,    7219.166,    7247.416, ...,    7364.584,
           7261.166,    7289.541],
       [   3576.875,    3576.875,    3576.875, ...,    3576.875,
           3576.875,    3576.875]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.25 359.5 359.75
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0


In [27]:
lats.shape

NameError: name 'lats' is not defined

Weighted means 

http://xarray.pydata.org/en/stable/examples/area_weighted_temperature.html



In [15]:
weights = np.cos(np.deg2rad(ds_2019.latitude))
weights.name = "weights"
print(weights)

<xarray.DataArray 'weights' (latitude: 721)>
array([-4.37113883e-08,  4.36335988e-03,  8.72656144e-03,  1.30895972e-02,
        1.74523834e-02,  2.18149554e-02,  2.61769947e-02,  3.05385347e-02,
        3.48994955e-02,  3.92597876e-02,  4.36194539e-02,  4.79781702e-02,
        5.23359738e-02,  5.66927791e-02,  6.10485077e-02,  6.54031932e-02,
        6.97565079e-02,  7.41085038e-02,  7.84590840e-02,  8.28081742e-02,
        8.71558040e-02,  9.15016532e-02,  9.58457589e-02,  1.00188047e-01,
        1.04528420e-01,  1.08866930e-01,  1.13203242e-01,  1.17537402e-01,
        1.21869326e-01,  1.26198918e-01,  1.30526245e-01,  1.34850949e-01,
        1.39173105e-01,  1.43492594e-01,  1.47809476e-01,  1.52123436e-01,
        1.56434491e-01,  1.60742566e-01,  1.65047571e-01,  1.69349566e-01,
        1.73648223e-01,  1.77943558e-01,  1.82235524e-01,  1.86524004e-01,
        1.90809056e-01,  1.95090353e-01,  1.99367940e-01,  2.03641742e-01,
        2.07911655e-01,  2.12177724e-01,  2.16439649e-0

In [21]:
str_weighted = ds_2019.str.mean(dim='time').weighted(weights)
weighted_mean = str_weighted.mean(("lon", "lat"))
print(weighted_mean)


<xarray.DataArray (latitude: 721, longitude: 1440)>
array([[-2605757.8, -2605757.8, -2605757.8, ..., -2605757.8, -2605757.8,
        -2605757.8],
       [-2576333.8, -2576286.8, -2576310.2, ..., -2576450.5, -2576427. ,
        -2576357. ],
       [-2586958.8, -2586818.8, -2586748.8, ..., -2587145.8, -2587005.8,
        -2586982.2],
       ...,
       [-3323008.8, -3323709.2, -3324550. , ..., -3321163.8, -3321724.8,
        -3322331.2],
       [-3262805.8, -3263272.8, -3263809.2, ..., -3261848.2, -3262152. ,
        -3262432. ],
       [-3132428.2, -3132428.2, -3132428.2, ..., -3132428.2, -3132428.2,
        -3132428.2]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.25 359.5 359.75
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0


In [30]:
print(weighted_mean.shape)

(721, 1440)


In [22]:
#plot data on other projections
plt.clf()

# t2m_test = ds.t2m.sel(time='1979-01-01', method='nearest')
# field_test=(ds_2019.sshf).sel(time=year_str+'-'+month_str+'-01', method='nearest')
field_test=weighted_mean #yearly average
# field_test=((ds_2019.sshf)/(31.*24.*3600.)).sel(time=year_str+'-'+month_str+'-01', method='nearest')
plt.figure()
ax = plt.axes(projection=ccrs.Mercator(central_longitude=180))
#     ax.stock_img()
ax.coastlines()

biasContDist=10
# iMinMax=-120
iMinMax=12.*biasContDist/2.+biasContDist/2.
# clevsTD=np.arange(-2*iMinMax,+0.0001,biasContDist)
clevsTD=np.arange(-2*iMinMax,0.0001,biasContDist)

fig = plt.figure(figsize=(18,12))
#     ax = plt.axes(projection=ccrs.Orthographic())
ax.coastlines()
ax.gridlines()
(field_test/86400.).plot.contourf(ax=ax, transform=ccrs.PlateCarree(), clevel=clevsTD, extend='both', cmap='coolwarm', vmin=-130, vmax=0, cbar_kwargs={'shrink': 0.4})


ax.set_title('Weighted Surface Net Thermal Radiation for 2019 (W/m^2)')


Text(0.5, 1.0, 'Weighted Surface Net Thermal Radiation for 2019 (W/m^2)')

<Figure size 864x504 with 0 Axes>

<Figure size 1296x864 with 0 Axes>

Create a xarray DataArray

https://unidata.github.io/python-training/workshop/XArray/xarray-introduction/

In [9]:

temp = np.cos(np.deg2rad(lats))
# temp = xr.DataArray(weights)
weights = xr.DataArray(temp, dims=['lats', 'lons'])
weights.attrs['standard_name'] = 'weights'

print(weights)

<xarray.DataArray (lats: 721, lons: 1440)>
array([[-4.371139e-08, -4.371139e-08, -4.371139e-08, ..., -4.371139e-08,
        -4.371139e-08, -4.371139e-08],
       [ 4.363360e-03,  4.363360e-03,  4.363360e-03, ...,  4.363360e-03,
         4.363360e-03,  4.363360e-03],
       [ 8.726561e-03,  8.726561e-03,  8.726561e-03, ...,  8.726561e-03,
         8.726561e-03,  8.726561e-03],
       ...,
       [ 8.726561e-03,  8.726561e-03,  8.726561e-03, ...,  8.726561e-03,
         8.726561e-03,  8.726561e-03],
       [ 4.363360e-03,  4.363360e-03,  4.363360e-03, ...,  4.363360e-03,
         4.363360e-03,  4.363360e-03],
       [-4.371139e-08, -4.371139e-08, -4.371139e-08, ..., -4.371139e-08,
        -4.371139e-08, -4.371139e-08]], dtype=float32)
Dimensions without coordinates: lats, lons
Attributes:
    standard_name:  weights


In [26]:
field_test=ds_2019.str.mean(dim='time')+ds_2019.ssr.mean(dim='time')+ds_2019.sshf.mean(dim='time')+ds_2019.slhf.mean(dim='time')
# (field_test/86400.).weighted(weights).mean(dim=('longitude')).plot()
(field_test/86400.).mean(dim=('longitude')).plot()
(ds_2019.ssr.mean(dim='time')/86400.).mean(dim=('longitude')).plot()
(ds_2019.str.mean(dim='time')/86400.).mean(dim=('longitude')).plot()
(1.*weighted_mean/86400.).mean(dim=('longitude')).plot()
(ds_2019.slhf.mean(dim='time')/86400.).mean(dim=('longitude')).plot()
(ds_2019.sshf.mean(dim='time')/86400.).mean(dim=('longitude')).plot()
labels = ['ssr','str','weighted_str','slhf','sshf']
plt.legend(labels,loc='lower right')
ax.legend()


2020-10-13 23:19:44,322 WARNING No handles with labels found to put in legend.


In [2]:
print(np.asarray(field_test)/86400))
# np.asarray(np.cos(np.pi*lats/180.)).shape

SyntaxError: unmatched ')' (<ipython-input-2-d824d1392a03>, line 1)

In [40]:
# field_test.mean(dim=('longitude'),dim=('latitude')))
print(np.mean(np.cos(np.pi*lats/180.)))
print(np.mean(np.asarray(np.cos(np.pi*lats/180.))*(np.asarray(field_test)/86400)))#/np.mean(np.cos(np.pi*lats/180.))
np.mean(np.asarray(field_test/86400))#/np.mean(np.cos(np.pi*lats/180.))

0.63573587


ValueError: shapes (721,1440) and (721,1440) not aligned: 1440 (dim 1) != 721 (dim 0)

In [1]:
1366/4.

341.5

In [2]:
1366/2.

683.0